# Timescale-vector

> Python library for storing vector data in Postgres

This file will become your README and also the index of your documentation.

## Install

```sh
pip install timescale_vector
```

## Basic Usage

Load up your postgres credentials. Safest way is with a .env file:

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os

In [ ]:
_ = load_dotenv(find_dotenv()) 
service_url  = os.environ['TIMESCALE_SERVICE_URL'] 

Next, create the client. 

This takes three arguments: 

* A connection string
* The name of the collection
* Number of dimensions

  In this tutorial, we will use the async client. But we have a sync client as well (with an almost identical interface)

In [ ]:
#| hide
import asyncpg

In [ ]:
#| hide
con = await asyncpg.connect(service_url)
await con.execute("DROP TABLE IF EXISTS my_data;")
await con.close()

In [ ]:
from timescale_vector import client 

In [ ]:
vec  = client.Async(service_url, "my_data", 2)

Next, create the tables for the collection:

In [ ]:
await vec.create_tables()

Next, insert some data. The data record contains:

* A uuid to uniquely identify the emedding
* A json blob of metadata about the embedding
* The text the embedding represents
* The embedding itself

Because this data already includes uuids we only allow upserts

In [ ]:
import uuid

In [ ]:
await vec.upsert([\
    (uuid.uuid4(), '''{"animal":"fox"}''', "the brown fox", [1.0,1.3]),\
    (uuid.uuid4(), '''{"animal":"fox", "action":"jump"}''', "jumped over the", [1.0,10.8]),\
])

Now you can query for similar items:

In [ ]:
await vec.search([1.0, 9.0])

[<Record id=UUID('0313cfac-07e4-4c01-9651-0917ecb1991c') metadata='{"action": "jump", "animal": "fox"}' contents='jumped over the' embedding=array([ 1. , 10.8], dtype=float32) distance=0.00016793422934946456>,
 <Record id=UUID('3bde3dd3-9445-4d9e-b72e-f329d19c380d') metadata='{"animal": "fox"}' contents='the brown fox' embedding=array([1. , 1.3], dtype=float32) distance=0.14489260377438218>]

You can specify the number of records to return.

In [ ]:
await vec.search([1.0, 9.0], k=1)

[<Record id=UUID('0313cfac-07e4-4c01-9651-0917ecb1991c') metadata='{"action": "jump", "animal": "fox"}' contents='jumped over the' embedding=array([ 1. , 10.8], dtype=float32) distance=0.00016793422934946456>]

You can also specify a filter on the metadata as a simple dictionary

In [ ]:
await vec.search([1.0, 9.0], k=1, filter={"action": "jump"})

[<Record id=UUID('0313cfac-07e4-4c01-9651-0917ecb1991c') metadata='{"action": "jump", "animal": "fox"}' contents='jumped over the' embedding=array([ 1. , 10.8], dtype=float32) distance=0.00016793422934946456>]

You can also specify a list of filter dictionaries, where an item is returned if it matches any dict

In [ ]:
await vec.search([1.0, 9.0], k=2, filter=[{"action": "jump"}, {"animal": "fox"}])

[<Record id=UUID('0313cfac-07e4-4c01-9651-0917ecb1991c') metadata='{"action": "jump", "animal": "fox"}' contents='jumped over the' embedding=array([ 1. , 10.8], dtype=float32) distance=0.00016793422934946456>,
 <Record id=UUID('3bde3dd3-9445-4d9e-b72e-f329d19c380d') metadata='{"animal": "fox"}' contents='the brown fox' embedding=array([1. , 1.3], dtype=float32) distance=0.14489260377438218>]

You can access the fields as follows

In [ ]:
records = await vec.search([1.0, 9.0], k=1, filter={"action": "jump"})
records[0][client.SEARCH_RESULT_ID_IDX]

UUID('d282ad19-1a69-4a9d-8a15-6f06262e109a')

In [ ]:
records[0][client.SEARCH_RESULT_METADATA_IDX]

'{"action": "jump", "animal": "fox"}'

In [ ]:
records[0][client.SEARCH_RESULT_CONTENTS_IDX]

'jumped over the'

In [ ]:
records[0][client.SEARCH_RESULT_EMBEDDING_IDX]

array([ 1. , 10.8], dtype=float32)

In [ ]:
records[0][client.SEARCH_RESULT_DISTANCE_IDX]

0.00016793422934946456

To delete all records use: 

In [ ]:
await vec.delete_all()

## Advanced Usage

### Indexing

Indexing speeds up queries over your data. 

By default, we setup indexes to query your data by the uuid and the metadata.

If you have many rows, you also need to setup an index on the embedding. You can create an ivfflat index with the following command after the table has been populated.

In [ ]:
await vec.create_ivfflat_index()

Please note it is very important to do this only after you have data in the table. 

You can drop the index with the following command.

In [ ]:
await vec.drop_embedding_index()

Please note the community is actively working on new indexing methods for embeddings. As they become available, we will add them to our client as well.